In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****IMPORTING IMPORTANT LIBRARIES****

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization,GlobalAveragePooling2D

# **EXAMPLE**

In [4]:
img_example=cv2.imread('../input/face-expression-recognition-dataset/images/train/angry/0.jpg')
plt.imshow(img_example)

**SHAPE OF IMAGE**

In [5]:
img_shape=img_example.shape
img_shape

**CLASSES**

In [6]:
classes=[]
count=[]
for i in os.listdir("../input/face-expression-recognition-dataset/images/train"):
    classes.append(i)
    count.append(len(os.listdir(os.path.join("../input/face-expression-recognition-dataset/images/train",i))))
    print(i,len(os.listdir(os.path.join("../input/face-expression-recognition-dataset/images/train",i))))


In [7]:
df1=pd.DataFrame()
df1['class']=classes
df1['count']=count
df1


**COUNTPLOT**

In [8]:
sns.catplot(data=df1,x="class",y="count",kind="bar")

# DATA AUGMENTATION

In [9]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=20,
                                                              data_format='channels_last',horizontal_flip=True,
                                                              width_shift_range=0.1,height_shift_range=0.1,
                                                              brightness_range=[0.2,1],shear_range=0.2,zoom_range=0.2)
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# LOADING DATA

In [10]:
train_datagen=train_datagen.flow_from_directory('../input/face-expression-recognition-dataset/images/train',target_size=(48,48),batch_size=64,class_mode='categorical',shuffle=True)
test_datagen=test_datagen.flow_from_directory('../input/face-expression-recognition-dataset/images/validation',target_size=(48,48),batch_size=64,class_mode='categorical',shuffle=False)


In [11]:
test_datagen.class_indices

# LOADING SAMPLE IMAGES OF EACH CLASS

In [12]:
fig=plt.figure(1, (14, 14))
cnt=0
for param in classes:
    cnt_copy=0
    plt.title(param)
    path=os.path.join('../input/face-expression-recognition-dataset/images/train',param)
    for img in os.listdir(path):
        if cnt_copy<7:
            cnt=cnt+1
            ax=plt.subplot(7,7,cnt)
            img=cv2.imread(os.path.join(path,img))
            ax.imshow(img)
            ax.set_title(param)
            plt.tight_layout()
            cnt_copy+=1
        else :
            break
            

In [13]:
batch_train=next(train_datagen)
type(train_datagen)

In [14]:
print(batch_train[0][0].shape)

In [15]:
type(batch_train[0])

In [16]:
type(batch_train)

**SAMPLE IMAGES IN A SHUFFLED BATCH**

In [17]:
plt.figure(figsize=(10,10))
for i in range(32):
    plt.subplot(8,4,i+1)
    plt.imshow(batch_train[0][i])
plt.show()    
    
    

# CHECKPOINTS

In [18]:
# save_callback=tf.keras.callbacks.ModelCheckpoint(filepath='./checkpoint',save_weights_only=False,monitor='val_loss',
#                                              save_best_only=True)
# checkpoint_path="./"
# checkpoint_dir = os.path.dirname(checkpoint_path)

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


early_stop= EarlyStopping(
                                monitor = "val_accuracy",
                                min_delta=0.001,
                                patience=5,
                                verbose=1,
                                mode="max")

reduce_lr = ReduceLROnPlateau(
                                monitor="val_accuracy",
                                factor=0.1,
                                patience=2,
                                verbose=1,
                                min_delta=0.0001)

**RESNET NET**

In [19]:
base_model=tf.keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(48,48,3))

model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(7,activation='softmax'))    

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

for layers in base_model.layers:
    layers.trainable=False

model.summary()

In [17]:
# train = train_datagen.cache().prefetch(tf.data.AUTOTUNE)
# val  = test_datagen.cache().prefetch(tf.data.AUTOTUNE)

**TRAINING MODEL**

In [18]:
history=model.fit_generator(train_datagen,validation_data=test_datagen,epochs=50,steps_per_epoch=train_datagen.n//train_datagen.batch_size,validation_steps=test_datagen.n//test_datagen.batch_size,verbose=1, callbacks=[early_stop,reduce_lr])

**SAVING MODEL**

In [19]:
tf.keras.models.save_model(model,'model.hdf5')

# PLOTTING ACCURACY AND LOSS CURVES

In [20]:
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(history.epoch, history.history['accuracy'], label='train')
sns.lineplot(history.epoch, history.history['val_accuracy'], label='valid')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(history.epoch, history.history['loss'], label='train')
sns.lineplot(history.epoch, history.history['val_loss'], label='valid')
plt.title('Loss')
plt.tight_layout()

plt.show()

In [21]:
predictions=model.predict(test_datagen, batch_size=64)
predicted_classes = np.argmax(predictions,axis=1)
true_labels=test_datagen.classes

# CONFUSION MATRIX

In [23]:
import scikitplot
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred=model.predict(test_datagen)
y_pred=np.argmax(y_pred,axis=1)

cm=confusion_matrix(test_datagen.classes,y_pred)

target_names=['angry','disgust','fear','happy','neutral','sad','surprise']

print(classification_report(test_datagen.classes,y_pred,target_names))

matrix=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=target_names)
matrix.plot()
plt.show()


In [21]:
mapper = {0:'angry',
 1:'disgust',
 2:'fear',
 3:'happy',
 4:'neutral',
 5:'sad',
 6:'surprise'}

sad_path="../input/face-expression-recognition-dataset/images/validation/sad"
neutral_path="../input/face-expression-recognition-dataset/images/validation/neutral"

neutral_filenames = np.random.choice([x for x in os.listdir(neutral_path) 
                                     if os.path.isfile(os.path.join(neutral_path, x))],size=9)

sad_filenames = np.random.choice([x for x in os.listdir(sad_path) 
                                     if os.path.isfile(os.path.join(sad_path, x))],size=9)


fig = plt.figure(1, (18, 4))

for i, (sadidx, neuidx) in enumerate(zip(sad_filenames, neutral_filenames)):
        ax = plt.subplot(2, 9, i+1)
        sadidx=os.path.join(sad_path,sadidx)
        sample_img = cv2.imread(sadidx)
        plt.imshow(sample_img)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"true:sad, pred:{mapper[np.argmax(model.predict(sample_img.reshape(1,48,48,3)))]}")

        ax = plt.subplot(2, 9, i+10)
        neuidx=os.path.join(neutral_path,neuidx)
        sample_img1 = cv2.imread(neuidx)
        ax.imshow(sample_img1)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"true:neut, pred:{mapper[np.argmax(model.predict(sample_img1.reshape(1,48,48,3)))]}")

        plt.tight_layout()


# **EFFICIENT NET**

In [1]:
from tensorflow.keras.applications.efficientnet import EfficientNetB2
base_model_efficientnet=EfficientNetB2(include_top=False,weights='imagenet',input_shape=(224,224,3))
# model1=Sequential()
# model1.add(base_model_efficientnet)
# model1.add(Flatten())
# model1.add(Dense(256,activation='relu'))
# model1.add(Dense(128,activation='relu'))
# model1.add(Dense(7,activation='softmax'))
x=Flatten()(base_model_efficientnet.output)
# x=Dense(256,activation='relu')(x)
# x = keras.layers.Dropout(0.1)(x)
# x=Dense(64,activation='relu')(x)
# x = keras.layers.Dropout(0.1)(x)
pred=Dense(7,activation='softmax')(x)
model1=Model(base_model_efficientnet.input,outputs=pred)
# when the layers arwe in a foood mooofa ehrlh ldf fhlsaa

for layers in base_model_efficientnet.layers:
    layers.trainable=False    
    
model1.summary()

In [ ]:
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model1.fit_generator(train_datagen,validation_data=test_datagen,epochs=1,steps_per_epoch=len(train_datagen),validation_steps=len(test_datagen),callbacks=[early_stop,reduce_lr])